# Task 1

A template code for calculating FCFs for $H_2$ - $H_2^+$ using the harmonic oscillator approximation. 

Imports:

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from FCF_helper import FCFSpec

Initialize the `FCFSpec` object. Modify `n_0_max` and `n_p_max` (the maximum vibrational states of $H_2$ and $H_2^+$ that we'll look at calculating FCFs for) in order to get 10 transitions starting from n_0=0.

In [2]:
n_0_max = 2
n_p_max = 2

fcfspec = FCFSpec(n_0_max, n_p_max)

Calculate all the FCFs and print the results.

In [3]:
spectrum = fcfspec.spectrum_analysis()
print("n_0\t\t n_p\t\t FCF")
print(spectrum)

n_0		 n_p		 FCF
[[0.         0.         1.        ]
 [0.         1.         2.95139081]
 [0.         2.         3.49048851]
 [1.         0.         1.55717552]
 [1.         1.         1.42278258]
 [1.         2.         0.00043658]
 [2.         0.         1.74174775]
 [2.         1.         0.34695781]
 [2.         2.         0.50038149]]
